In [8]:
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit

from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Statevector
import os
from qiskit.quantum_info import Pauli
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import HGate , SdgGate
from qiskit import transpile


/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [15]:
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# 1. Pauli 문자열 목록
paulis = ['YYYY','XXXX',"XXYY","YYXX"]

# 2. 계수 리스트
coeffs = np.array([1,1,1,1], dtype=complex)

# 3. SparsePauliOp 생성
hamiltonian = SparsePauliOp(paulis, coeffs=coeffs)
H_matrix  = hamiltonian.to_matrix()
print(hamiltonian)
print(H_matrix)




SparsePauliOp(['YYYY', 'XXXX', 'XXYY', 'YYXX'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  4.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 4.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0

In [10]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()
  qubit_op = mapper.map(second_q_op)
  return qubit_op , mapper
# 고전값 비교용 FCI 
# H2 인경우로만 해뒀고, 거리 주면 에너지 계산할 수 있음. 

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

def cost_func(parameter, H_matrix, n_ptl, ansatz, backend,k):
    params = ansatz.parameters 
    param_dict = dict(zip(params, parameter))
    qc = ansatz.assign_parameters(param_dict)
    qc.measure_all()
    job = backend.run(qc, shots=2000)
    result = job.get_probabilities()
    all_basis = []
    all_prob = []
    for bitstring, prob in result.items():
        count_ones = bitstring.count('1')
        half = len(bitstring) // 2
        left_ones = bitstring[:half].count('1')
        right_ones = bitstring[half:].count('1')
        if count_ones == n_ptl and left_ones==right_ones :
            all_basis.append(bitstring)
            all_prob.append(prob)

    all_basis = np.array(all_basis)
    all_prob = np.array(all_prob)
    top_indices = np.argsort(-all_prob)[:k]
    probable_basis = all_basis[top_indices]
    print("most probable_basis : ", probable_basis)

    n = len(probable_basis)
    H = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            basis_i = probable_basis[i]
            basis_j = probable_basis[j]
            sv1 = Statevector.from_label(basis_i)
            sv2 = Statevector.from_label(basis_j)
            inner_product = sv1.conjugate().data @ (H_matrix @ sv2.data)
            H_ij = np.real(inner_product)
            H[i, j] = H_ij 
            print("<{}|H|{}> = {}".format(basis_i,basis_j,H_ij))
    eigvals, eigvecs = np.linalg.eig(H)
    Energy = np.min(eigvals)

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(Energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {Energy}]")

    return Energy

In [3]:
api_key = "7IMO85iuzmzAAgri4osG5BPTeCFXrKoM"
provider = IonQProvider(api_key)
simulator_backend = provider.get_backend("simulator")
#simulator_backend = provider.get_backend("qpu.forte-1")

In [11]:
atoms = ["H", "H"]
basis = 'sto3g'
dist = 0.735
coords = [(0,0,0), (dist,0,0)]
charge = 0
multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 

fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
print(second_q_op)
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
H_matrix  = hamiltonian.to_matrix()
print(hamiltonian)
print(H_matrix)

Fermionic Operator
number spin orbitals=4, number terms=36
  0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.3322908651276482 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.3322908651276482 * ( +_0 +_3 -_3 -_0 )
+ 0.0904655998921157 * ( +_0 +_0 -_1 -_1 )
+ 0.0904655998921157 * ( +_0 +_1 -_0 -_1 )
+ 0.0904655998921157 * ( +_0 +_2 -_3 -_1 )
+ 0.0904655998921157 * ( +_0 +_3 -_2 -_1 )
+ 0.0904655998921157 * ( +_1 +_0 -_1 -_0 )
+ 0.0904655998921157 * ( +_1 +_1 -_0 -_0 )
+ 0.0904655998921157 * ( +_1 +_2 -_3 -_0 )
+ 0.0904655998921157 * ( +_1 +_3 -_2 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.34928686136600906 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.34928686136600906 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.3322908651276482 * ( +_2 +_1 -_1 -_2 )
+ 0.33785507740175813 * ( +_2 +_2 -_2 -_2 )
+ 0.3322908651276482 * ( +_2 +_3 -_3 -_2 )
+ 0.0904655998921157 * ( +_2 +_0 -_1 -_3 )
+ 0.0904655998921157 * ( +_2 +_1

In [4]:
atoms = ["H", "H"]
basis = 'sto3g'
dist = 0.735
coords = [(0,0,0), (dist,0,0)]
charge = 0
multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 

fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
print(second_q_op)
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
H_op = hamiltonian.to_operator()
H_matrix = H_op.data
num_qubits = hamiltonian.num_qubits
num_particles = E_problem.num_particles
num_electrons = np.sum(num_particles)
num_spatial_orbitals = E_problem.num_spatial_orbitals


init_state = HartreeFock(num_spatial_orbitals,num_particles,mapper)
ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state).decompose()
#ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)

pauli_basis = hamiltonian.paulis
coeffs = hamiltonian.coeffs
num_params = ansatz.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)

res = minimize(
    cost_func,
    x0,
    args=(H_matrix, num_electrons, ansatz, simulator_backend,3),
    method="cobyla",
)



plt.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"] + repulsion, label='E_VQE')
plt.xlabel("Iterations (#)")
plt.ylabel("Energy (Hartree)")
plt.title("Energy Calc. for each iteration")
plt.legend()
plt.grid()
plt.draw()

most probable_basis :  ['1010' '0101' '0110']
<1010|H|1010> = -0.2452182918302661
<1010|H|0101> = 0.18093119978423136
<1010|H|0110> = 0.0
<0101|H|1010> = 0.18093119978423136
<0101|H|0101> = -1.8369679912029833
<0101|H|0110> = 0.0
<0110|H|1010> = 0.0
<0110|H|0101> = 0.0
<0110|H|0110> = -1.0636533500290954
Iters. done: 1 [Current cost: -1.857275030202379]
most probable_basis :  ['0110' '1010' '0101']
<0110|H|0110> = -1.0636533500290954
<0110|H|1010> = 0.0
<0110|H|0101> = 0.0
<1010|H|0110> = 0.0
<1010|H|1010> = -0.2452182918302661
<1010|H|0101> = 0.18093119978423136
<0101|H|0110> = 0.0
<0101|H|1010> = 0.18093119978423136
<0101|H|0101> = -1.8369679912029833
Iters. done: 2 [Current cost: -1.8572750302023793]
most probable_basis :  ['0110' '1001' '1010']
<0110|H|0110> = -1.0636533500290954
<0110|H|1001> = 0.18093119978423136
<0110|H|1010> = 0.0
<1001|H|0110> = 0.18093119978423136
<1001|H|1001> = -1.0636533500290954
<1001|H|1010> = 0.0
<1010|H|0110> = 0.0
<1010|H|1001> = 0.0
<1010|H|1010> = -

Traceback (most recent call last):
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
    f = sf.fun(x)
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 278, in fun
    self._update_fun()
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 262, in _update_fun
    self._update_fun_impl()
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 163, in update_fun
    self.f = fun_wrapped(self.x)
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 145, 

: 

: 